# Requests+beautifulsoup4

In [ ]:
import requests

r=requests.get('https://en.wikipedia.org/')

In [ ]:
r.ok

In [ ]:
r

2XX成功/3XX重新導向/4XX用戶端錯誤/5XX伺服器端錯誤

In [ ]:
print(len(r.text))

In [ ]:
from bs4 import BeautifulSoup

soup=BeautifulSoup(r.text,'html.parser')

還有'lxml' 'html5lib' 等剖析器

抓title

In [ ]:
print(soup.title)

抓<a href='OOXX'>...<a>內所有超連結OOXX

In [ ]:
for link in soup.find_all('a'):
    print(link.get('href'))

抓div內特定class

In [ ]:
soup.find_all('div',class_='otd-footer')

抓 a tag下特定title

In [ ]:
soup.find('a',title='Special:Statistics').contents

## 蘋果賽馬網(網站已改版，無法用一般方法查詢)

In [ ]:
import requests
from bs4 import BeautifulSoup

url='https://racing.appledaily.com.hk/race-day/race?raceDay=2691'

r=requests.get(url)

soup=BeautifulSoup(r.text,'html.parser')
print(soup.title)

In [ ]:
soup.find_all('select')

In [ ]:
import time
import re

race_number=23795
race_list=[]

for i in range(8):
    temp=str(race_number)
    race_list.append('https://racing.appledaily.com.hk/race-day/race?raceDay=2691'+f'&race={temp}')
    race_number+=1

In [ ]:
race_list

In [ ]:
import pandas as pd
import html5lib
data=pd.read_html('https://racing.appledaily.com.hk/race-day/race?raceDay=2691&race=23795')
print(len(data))

## 蝦皮購物

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://shopee.tw/search?keyword=ps4%20pro%20%E4%B8%BB%E6%A9%9F'
r=requests.get(url,allow_redirects=False)
print(r.status_code)
print(len(r.text))

In [ ]:
soup=BeautifulSoup(r.text,'html.parser')
items=soup.find_all('div', class_="col-xs-2-4 shopee-search-item-result__item")
print(len(items))

有些網站會在中途把使用者重新導向，因此要加 allow_redirects=False

In [ ]:
print(r.history)
print(r.url)

使用useragent告訴server你的身分，避免被重新導向

In [ ]:
from fake_useragent import UserAgent
ua=UserAgent()
for i in range(5):
    print(ua.random)
    print('*------*')


有些網站會有 robots.txt 寫可使用的User-Agent和其他規定

In [ ]:
headers={
    'User-Agent':'Googlebot'
}
r=requests.get(url,headers=headers,allow_redirects=True)
print(r.status_code)
print(r.history)
print(r.url)

soup=BeautifulSoup(r.text,'html.parser')
items=soup.find_all('div', class_='col-xs-2-4 shopee-search-item-result__item')
print(len(items))

有些商品prices有區間，因此下面程式有錯

In [ ]:
contents=soup.find_all('div',class_='_1NoI8_')
prices=soup.find_all('span',class_='_341bF0')
all_items=soup.find_all('div',class_='col-xs-2-4 shopee-search-item-result__item')
links=[i.find('a').get('href') for i in all_items]

for c,p,l in zip(contents,prices,links):
    print(c.contents[0])
    print(p.contents[0])
    print('https://shopee.tw/'+l)
    print('*------------------*')

### 完整程式碼(prices 一樣有錯)

In [ ]:
import requests
from bs4 import BeautifulSoup

def shopee_scraper(keyword,n_page=0,used=False,new=False):
    '''
    參數說明:
        keyword: 商品名稱關鍵字
        n_page: 第幾頁(每頁有50個商品)
        used: 是否為二手商品?
        new: 是否為新商品?
    '''
    url = f'https://shopee.tw/search?keyword={keyword}&page={n_page}&sortBy=relevancy'
    if used:
        url += '&newItem=true'
    if new:
        url += '&usedItem=true'
    
    headers = {
        'User-Agent': 'Googlebot',
        #'From': 'YOUR EMAIL ADDRESS'
    }
    
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    contents = soup.find_all("div", class_="_1NoI8_")
    prices = soup.find_all("span", class_="_341bF0")
    all_items = soup.find_all("div", class_="col-xs-2-4 shopee-search-item-result__item")
    links = [i.find('a').get('href') for i in all_items]
    
    for c, p, l in zip(contents, prices, links):
        print(c.contents[0])
        print(p.contents[0])
        print('https://shopee.tw/'+l)
        print('*---------------------------------*')

In [ ]:
shopee_scraper('ipad air',0)

## 爬政府電子採購網

In [ ]:
import requests

url = 'https://web.pcc.gov.tw/prkms/prms-searchBulletionClient.do?root=tps'

data = {'tmpQuerySentence': None,
'timeRange':'109/1/1-109/12/31',
'querySentence':'新北市政府',
'tenderStatusType':'決標',
'sortCol':'AWARD_NOTICE_DATE',
'timeRangeTemp':'109/1/1-109/12/31',
'sym':'on',
'itemPerPage':'10'}

r = requests.post(url, data)
print(r.ok)
print(r.is_redirect)

In [ ]:
import re

patterns=['style="width:18%;text-align:left">(.*?)</td>','<div class="wordwrap">(.*?)</div>','style="width:9%;text-align:left;min-width:8%;">(.*?)</td>','style="width:15%;text-align:left;min-width:40px;">(.*?)</td>','href="(.*?pkAtmMain.*?)"']

agency_name = re.findall(patterns[0], r.text)  # 機關名稱
project_name = re.findall(patterns[1], r.text)  # 標案名稱
project_announce_date = re.findall(patterns[2], r.text)  #標案公告日期
FF_date_t = re.findall(patterns[3], r.text)  
FF_date = [re.findall('\d\d\d\/\d\d\/\d\d',i)[0] for i in FF_date_t] # 決標或無法決標日期
Success = ['無法決標' not in i for i in FF_date_t] # 是否得標
Detail_url = re.findall(patterns[4], r.text) # 詳細頁面的網址

In [ ]:
detail_content = []
for b, d_url in zip(Success, Detail_url):
    if b:
        r = requests.get('https://web.pcc.gov.tw' + d_url)
        if len(r.history) > 0:
            print(f'You are redirected to: {r.url}')
            break
        detail_content.append(r.text)
        time.sleep(1)

In [ ]:
import requests
import re
import pandas as pd

def GetTenderData(start_date, end_date, keyword, max_items):
   
    url = 'https://web.pcc.gov.tw/prkms/prms-searchBulletionClient.do?root=tps'
    
    data = {'tmpQuerySentence': None,
    'timeRange':f'{start_date}-{end_date}',
    'querySentence':f'{keyword}',
    'tenderStatusType':'決標',
    'sortCol':'TENDER_NOTICE_DATE',
    'timeRangeTemp':f'{start_date}-{end_date}',
    'd-7095067-p':'1',
    'sym':'on',
    'itemPerPage':f'{max_items}'}
    
    r = requests.post(url,data = data)

    patterns = [
            'style="width:18%;text-align:left">(.*?)</td>',
            '<div class="wordwrap">(.*?)</div>',
            '</a>[\s\S]*?width:9%;text-align:left;min-width:8%;">(.*?)</td>',
            '"width:15%;text-align:left;min-width:40px;">([\s\S]*?)</td>',
            'href="(.*?pkAtmMain.*?)"'
            ]
    agency_name = re.findall(patterns[0], r.text)
    project_name = re.findall(patterns[1], r.text)
    project_announce_date = re.findall(patterns[2], r.text)
    FF_date_t = re.findall(patterns[3], r.text)
    FF_date = [re.findall('\d\d\d\/\d\d\/\d\d',i)[0] for i in FF_date_t]
    Success = ['無法決標' not in i for i in FF_date_t]
    Detail_url = ['https://web.pcc.gov.tw' + u for u in re.findall(patterns[4], r.text)]
    
    col_names = ['機關名稱','標案名稱','標案公告日期','決標或無法決標日期','是否決標?','標案網址']
    
    df = pd.DataFrame([agency_name, project_name, project_announce_date,FF_date,Success,Detail_url]).T
    df.columns = col_names
    df.to_excel('政府電子採購網資料.xlsx',index=False)

In [ ]:
GetTenderData('109/01/01','109/12/31','新北市政府',100)